In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import cv2
import numpy as np
import os
import re
import shutil

import keras

from models import BachNet
from models import ChopinNet

import networkx as nx
import sys
import time


from heapq import heappop as pop
from heapq import heappush as push
from utils import graph_utils
from utils import display_utils
from utils import prediction_utils
from utils import preprocessing_utils

Using TensorFlow backend.


In [4]:
train_path = "data/train"
test_path = "data/test"

input_path = "input"
output_path = "output"

gt_tag = "gt"

receptive_field_shape = (12, 12)
n_epochs = 32
save_rate = 2

In [5]:
bach = BachNet.BachNet()
bach.build(receptive_field_shape, 1)
bach.load_model('models/saved_models/Bach/model.h5')

batch = dict()
input_gen = prediction_utils.input_generator(bach, train_path, input_path, gt_tag)

while True:
    try:
        f_name, img, bps, I_a, gt, gt_cuts, seeds = next(input_gen)
        graph = graph_utils.prims_initialize(img)
        batch[f_name] = img, bps, I_a, gt, gt_cuts, seeds, graph
    except StopIteration:
        break

Could not find model. Creating a new one.
Loading image:  segprob.0083_gt
Loading image:  segprob.0083
Loading image:  segprob.0083
Loading image:  segprob
Loading image:  segprob
Loading image:  segprob_gt


In [6]:
chopin = ChopinNet.Chopin()
chopin.build(receptive_field_shape, learning_rate=1e-5)
#chopin.load_model("models/saved_model/Chopin/checkpoint")
chopin.initialize_session()

In [7]:
global_loss_timeline = []
loss_timelines = dict()
loss_file = open("data/train/chopin/global_loss.txt", 'w')
loss_file.write("f_name\tepoch\tloss\n")

for f_name, (img, bps, I_a, gt, gt_cuts, seeds, graph) in batch.iteritems():
    print("Training on", f_name)
    foldername = os.path.join(train_path, "chopin", f_name)
    for epoch in range(n_epochs):
        print("Epoch: {}".format(epoch))
        loss, segmentations, cuts = chopin.train_on_image(img, I_a, gt_cuts, seeds, graph)
        
        if epoch % save_rate:
            print("Saving Model")
            chopin.save_model(os.path.join(foldername, "saved_models", "model"), epoch)
            chopin.save_model("models/saved_models/Chopin/chopin", epoch)
        
        plt.imsave(os.path.join(foldername, "epoch_{}_bw.png".format(epoch)), display_utils.view_boundaries(np.zeros_like(img), cuts))

        mask = display_utils.transparent_mask(display_utils.view_boundaries(img, gt_cuts), segmentations, alpha=0.75)
        plt.imsave(os.path.join(foldername, "epoch_{}_overlay.png".format(epoch)), mask)
        
        loss_file.write(f_name + "\t" + str(epoch) + "\t" + str(loss) + "\n")
        loss_file.flush()
        
        global_loss_timeline.append(loss)
        
        try:
            loss_timelines[f_name].append(loss)
        except KeyError:
            loss_timelines[f_name] = [loss]
            
        plt.plot(loss_timelines[f_name])
        plt.savefig(os.path.join(foldername, "local_loss"))
        
        plt.gcf().clear()
            
        plt.plot(global_loss_timeline)
        plt.savefig("data/train/chopin/global_loss")
        
        plt.gcf().clear()
            
loss_file.close()

Training on segprob
Epoch: 0
Starting gradient segmentation...
Segmentation done: 204.813649s
Starting gradient segmentation...
Segmentation done: 0.899932s
Epoch: 1
Starting gradient segmentation...


KeyboardInterrupt: 

In [15]:
foldername

'data/train/chopin/740_1460'

In [16]:
chopin.save_model(foldername)

In [10]:
os.mkdir(os.path.join(foldername, "saved_models"))

In [9]:
os.path.exists(os.path.join(foldername, "saved_models"))

True

In [10]:
os.path.join(foldername, "saved_models", str(epoch))

'data/train/chopin/740_1460/saved_models/1'